In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from typing import Annotated, Iterable, Optional, TypedDict
from polaroids import DataFrame, Field
from polaroids.dataframe import _Metadata
import polars as pl
from polaroids import _utils

from typing import Annotated, TypedDict
from polaroids import DataFrame, Field
from polaroids.types import int32, float64
import polars as pl

class Schema(TypedDict):
    a: Annotated[list[int32 | None], Field()]
    b: Annotated[list[list[bool]], Field()]
    c: Annotated[list[int] | None, Field()]
    d: Optional[int]
    e: int32 | None
    f: int | None

# _utils.typeddict_to_polats_schema(Schema)


raw = (
    pl.DataFrame({"a": [0.0, 1.0], "b": [None, 0]})
    .pipe(DataFrame[Schema])
)
# df = (
#     raw
#     .sort("a")
#      # <- Add a Schema to your dataframe
#     # .select(pl.all())
#     .validate()
#     # .rows()
#     # .sort("a")
# )


In [3]:
from typing import Literal

from polaroids._parse_types import typeddict_to_polats_schema

class SubSchema(TypedDict):
    a: int
    b: str

class Schema(TypedDict):
    a: Annotated[list[int32 | None], Field()]
    b: Annotated[list[list[bool]], Field()]
    c: Annotated[list[int] | None, Field()]
    d: Optional[int]
    e: int32 | None
    f: int | None
    g: Literal["a", "b"]
    h: list[Literal["a", "b"] | None]

expected = pl.Schema({
    "a": pl.List(pl.Int32),
    "b": pl.List(pl.List(pl.Boolean)),
    "c": pl.List(pl.Int64),
    "d": pl.Int64,
    "e": pl.Int32,
    "f": pl.Int64,
    "g": pl.Enum(["a", "b"]),
    "h": pl.List(pl.Enum(["a", "b"])),
}) 
assert expected == typeddict_to_polats_schema(Schema)

In [24]:
class SubSchema(TypedDict):
    a: int
    b: str

class Schema(TypedDict):
    s: list[SubSchema]
    t: int

expected = pl.Schema([('s', pl.List(pl.Struct({'a': pl.Int64, 'b': pl.String}))), ('t', pl.Int64)])
assert expected == typeddict_to_polats_schema(Schema)

In [ ]:
from gc import set_debug
from typing import Annotated, TypedDict
from polaroids import DataFrame, Field
import polars as pl

class SubSchema(TypedDict):
    c: list[bool]
    d: str

class Schema(TypedDict):
    a: Annotated[int, Field(
        sorted="ascending",
        coerce=True,
        unique=True,
        checks=[lambda d: d.ge(0)],
    )]
    b: int | None
    s: SubSchema

df = (
    pl.DataFrame({
        "a": [0.0, 1.0], 
        "b": [None, 0], 
        "s": [{"c": [True], "d": "0"}, {"c": [True, False], "d": "1"}]
    })   
    .pipe(DataFrame[Schema]) # <- Add a Schema to your dataframe
    .validate() # Validate it from the Schema annotations!
)
shape: (2, 3)
┌─────┬──────┬─────────────────────┐
│ a   ┆ b    ┆ s                   │
│ --- ┆ ---  ┆ ---                 │
│ i64 ┆ i64  ┆ struct[2]           │
╞═════╪══════╪═════════════════════╡
│ 0   ┆ null ┆ {[true],"0"}        │
│ 1   ┆ 0    ┆ {[true, false],"1"} │
└─────┴──────┴─────────────────────┘

shape: (2, 3)
┌─────┬──────┬─────────────────────┐
│ a   ┆ b    ┆ s                   │
│ --- ┆ ---  ┆ ---                 │
│ i64 ┆ i64  ┆ struct[2]           │
╞═════╪══════╪═════════════════════╡
│ 0   ┆ null ┆ {[true],"0"}        │
│ 1   ┆ 0    ┆ {[true, false],"1"} │
└─────┴──────┴─────────────────────┘


In [ ]:
row = pl.DataFrame(df).row(0, named=True)
a: bool = row["s"]["c"][0]
# row["not_exists"]


In [33]:
from typing import Any, reveal_type


a : dict[str, int]= {"a": 1}
b = a["a"]
reveal_type(b)


Runtime type is 'int'


1

In [ ]:
dict(typeddict_to_polats_schema(Schema))

In [ ]:
from typing import get_type_hints, is_typeddict
import typing


input = Schema.__annotations__["s"]
is_typeddict(input)

In [ ]:
pl.Struct(pl.Schema({"a": pl.Int16}))

In [ ]:
input.__annotations__

In [ ]:
from polaroids._parse_types import parse_into_dtype


parse_into_dtype(input.__annotations__)

In [ ]:
a = SubSchema()
type(a)

In [ ]:
_utils.typeddict_to_polats_schema(Schema)

In [ ]:
pl.Series("a", [[1, 2], [4, 3]], dtype=pl.Array(pl.Int64, 2)).to_frame().rows()

In [ ]:
_utils.typeddict_to_polats_schema(Schema)

In [ ]:
_utils.typeddict_to_polats_schema(Schema)

In [ ]:
input.__origin__
input.__args__

In [ ]:
raw._schema

In [ ]:
pl.U

In [ ]:
pl.Series([], dtype=dict[str, str])

In [ ]:
pl.Str

In [ ]:
from types import UnionType
from typing import ForwardRef, Union, get_type_hints

UnionTypeOld = type(Union[int, str])

isinstance(get_type_hints(Schema)["b"], ForwardRef)
isinstance(input, (UnionType, UnionTypeOld))

In [ ]:
int32.__forward_arg__

In [ ]:
Schema.__annotations__["a"].__forward_arg__

In [ ]:
from polars.datatypes._parse import parse_py_type_into_dtype

__forward_arg__


In [ ]:
import enum
from inspect import isclass
import sys
from typing import TYPE_CHECKING, Any, NoReturn
from polars.datatypes.classes import (
    Binary,
    Boolean,
    Date,
    Datetime,
    Decimal,
    Duration,
    Enum,
    Float64,
    Int64,
    List,
    Null,
    Object,
    String,
    Time,
    Unknown,
)

from datetime import date, datetime, time, timedelta
from decimal import Decimal as PyDecimal

from polars.datatypes.convert import is_polars_dtype

from polars._typing import PolarsDataType, PythonDataType, SchemaDict

from types import NoneType, UnionType


def parse_py_type_into_dtype(input: PythonDataType | type[object]) -> PolarsDataType:
    """Convert Python data type to Polars data type."""
    if input is int:
        return Int64()
    elif input is float:
        return Float64()
    elif input is str:
        return String()
    elif input is bool:
        return Boolean()
    elif isinstance(input, type) and issubclass(input, datetime):  # type: ignore[redundant-expr]
        return Datetime("us")
    elif isinstance(input, type) and issubclass(input, date):  # type: ignore[redundant-expr]
        return Date()
    elif input is timedelta:
        return Duration
    elif input is time:
        return Time()
    elif input is PyDecimal:
        return Decimal
    elif input is bytes:
        return Binary()
    elif input is object:
        return Object()
    elif input is NoneType:
        return Null()
    elif input is list or input is tuple:
        return List
    elif isclass(input) and issubclass(input, enum.Enum):
        return Enum(input)
    # this is required as pass through. Don't remove
    elif input == Unknown:
        return Unknown
    elif hasattr(input, "__origin__") and hasattr(input, "__args__"):
        return _parse_generic_into_dtype(input)
    # -- Custom code --
    elif input.__module__ == "polaroids.types":
        return _utils.annotation_to_polars_dtype(input)
    # -- Custom code --
    else:
        _raise_on_invalid_dtype(input)


def _parse_generic_into_dtype(input: Any) -> PolarsDataType:
    """Parse a generic type (from typing annotation) into a Polars data type."""
    base_type = input.__origin__
    if base_type not in (tuple, list):
        _raise_on_invalid_dtype(input)

    inner_types = input.__args__
    inner_type = inner_types[0]
    if len(inner_types) > 1:
        all_equal = all(t in (inner_type, ...) for t in inner_types)
        if not all_equal:
            _raise_on_invalid_dtype(input)

    inner_type = inner_types[0]
    inner_dtype = parse_py_type_into_dtype(inner_type)
    return List(inner_dtype)


def _raise_on_invalid_dtype(input: Any) -> NoReturn:
    """Raise an informative error if the input could not be parsed."""
    input_type = input if type(input) is type else f"of type {type(input).__name__!r}"
    input_detail = "" if type(input) is type else f" (given: {input!r})"
    msg = f"cannot parse input {input_type} into Polars data type{input_detail}"
    raise TypeError(msg) from None


In [ ]:
parse_py_type_into_dtype(int32)

In [ ]:
raw._schema

In [ ]:
from typing_extensions import get_type_hints


t = get_type_hints(Schema)["b"]

In [ ]:
t.__module__

In [ ]:
pl.datatypes.dtype_str_to_dtype("int32")

In [ ]:
pl.DataType.from_python("int")

In [ ]:
column, checks = (
    self
    ._metadata
    .filter(pl.col.checks.is_not_null())
    .select("column", "checks")
    .row(0)
)

result = raw.select([
    check(pl.col(column)).alias(str(i))
    for i, check in enumerate(checks)
])
for i, check_ok in result.select(pl.all().all()).row(0, named=True).items():
    if not check_ok:
        df_failure = raw.filter(result.get_column(i))

In [ ]:
raw.with_columns(result.get_column(i).alias("__polaroids_tmp__")).fil

In [ ]:
result.get_column(i).arg_true()

In [ ]:
result.select(pl.all().all()).transpose(include_header = True, column_names = ["check_ok"])

In [ ]:
df = pl.DataFrame({"a": [0.0, 1.0], "b": [None, 0]})
DataFrame[Schema](df)._metadata

In [ ]:
hasattr(pl.DataFrame, "validate")

In [ ]:
print(getattr(df, "__orig_class__", 0))

In [ ]:
df._typeddict

In [ ]:
df.__orig_class__

In [ ]:
type(df).__orig_bases__[0]

In [ ]:
getattr(df, "__orig_class__", type(df).__orig_bases__[0])

In [ ]:
type(df).__orig_bases__[0].__args__

In [ ]:
pl.DataFrame(pl.DataFrame())

In [ ]:
(
    pl.DataFrame(df).__getattribute__("validate")
    
)

In [ ]:
pl.DataFrame.head

In [ ]:
row = df.row(0, named=True)
row["a"]
row["not_exists"]




In [ ]:


from typing import Annotated, Self, TypedDict
from polaroids import DataFrame, Field
import polars as pl

class BasicSchema(TypedDict):
    a: Annotated[pl.Int64, Field(
        sorted="ascending",
        coerce=True,
        unique=True,
        checks=[lambda d: d.ge(0)],
    )]
    b: int | None

df = pl.DataFrame({"a": [0.0, 1.0], "b": [None, 0]})

df_validated = df.pipe(DataFrame[BasicSchema]).validate()
df_validated
# shape: (2, 2)
# ┌─────┬──────┐
# │ a   ┆ b    │
# │ --- ┆ ---  │
# │ i64 ┆ i64  │
# ╞═════╪══════╡
# │ 0   ┆ null │
# │ 1   ┆ 10   │
# └─────┴──────┘


class BasicSchemaDataFrame(DataFrame[BasicSchema]):
    def check_a_greater_then_b(self) -> None:
        assert self.select((pl.col("a") >= pl.col("b")).all()).item(), "a should be greater the b"

# Example usage
BasicSchemaDataFrame(df).validate() # Passes validation

# This will raise an AssertionError
(
    pl.DataFrame({"a": [5, 6], "b": [None, 10]})
    .pipe(BasicSchemaDataFrame)
    .validate() # This will raise 💣 !
)


In [ ]:
print(df_validated)

In [ ]:
from typing import TypedDict
from polaroids import DataFrame, Field
import polars as pl

class BasicSchema(TypedDict):
    a: Annotated[pl.Int64, Field(
        sorted="ascending",
        coerce=True,
        unique=True,
        checks=[lambda d: d.ge(0)],
    )]
    b: str | None

print(
    pl.DataFrame({"a": [2.0, 5.0], "b": ["a", None]})
    .pipe(DataFrame[BasicSchema])
    .validate()
)

In [ ]:


from collections.abc import Callable

from polars import String



df = (
    pl.DataFrame({"a": [0,None,2,3], "b": [None,0,0,0], "c": [-1.0, 2, 4,10]})
    .pipe(DataFrame[BasicSchema])
    # ._typeddict
    # .__annotations__
    # .set_sorted("a")
    # .__annotations__
    # ._metadata
    .validate()
    # .schema
    # ._metadata
    # .validate()
    # .pipe(pl.DataFrame).select("sorted", "column")
    # .schema["sorted"]
    # .pipe(lambda d: d.to_dict())
    # .group_by("sorted").agg(pl.col("column"))
    # .group_by("sorted").agg(pl.col("column"))
    # .filter(pl.col("unique"))["column"].to_list()
    
    # ["checks"] 
    # .__args__[0].__origin__ is Callable
)
df

In [ ]:
from ctypes import util


_utils.typeddict_to_polats_schema(_Metadata)

In [ ]:
from typing import get_type_hints

class BasicSchema(TypedDict):
    a: Annotated[pl.Int64, Field(
        sorted="ascending",
        coerce=True,
        unique=True,
        checks=[lambda d: d.ge(0)],
    )]
    b: Annotated[pl.Int64, Field(primary_key=False)]
    c: Annotated[int | None, Field(coerce=True)]

get_type_hints(BasicSchema)
# 'a': Int64, 'b': Int64, 'c': int | None}

# I want to retrieve the nullable columns 
# here : nullable_cols == ["c"] 
nullable_cols = [name for name, hint in get_type_hints(BasicSchema).items() if type(None) in getattr(hint, "__args__", [])]
nullable_cols

In [ ]:
type(None)

In [ ]:
None.__class__

In [ ]:
pl.DataFrame({"sorted": ["ascending"], "column": ["a"]}, schema_overrides={"sorted": e}).group_by("sorted").agg(pl.col("column"))

In [ ]:
df = pl.DataFrame({'A' : [1,2,3]})
df = df.set_sorted("A")
df["A"].flags
# (
#     pl.DataFrame({'A' : [1,2,3]})
#     .with_columns(pl.col('A').set_sorted())
#     # .write_parquet('tmp.parquet')
# )

# pl.read_parquet('tmp.parquet')['A'].flags 
# Today: {'SORTED_ASC': False, 'SORTED_DESC': False}
# Wish: {'SORTED_ASC': True, 'SORTED_DESC': False}

In [ ]:
(
    pl.DataFrame({"a": [0,1,2,3], "b": [0,0,0,0]})
    .select(pl.all().is_duplicated().any())
    .transpose(include_header = True, column_names = ["is_duplicated"])
    .filter(pl.col("is_duplicated")).get_column("column").to_list()
)

In [ ]:
(
    pl.DataFrame({"a": [0,1,2,3], "b": [0,0,0,0]})
    
    # .select(pl.all().filter(pl.element().is_duplicated()).implode())
    # .select(pl.all().is_unique().all())
    # .transpose(include_header = True, column_names = ["is_unique"])
    # .filter(~pl.col("is_unique")).get_column("column").to_list()
)